In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
from PIL import Image
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc, roc_curve
from tqdm import tqdm

import time

In [3]:
# Keras & Tensorflow
import keras
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Lambda, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D , Conv2D , MaxPooling2D
from keras.layers import  Dropout , BatchNormalization , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy

In [4]:
benign_train = np.load('/content/drive/MyDrive/CSE499/numpy/benign_train.npy')
malign_train = np.load('/content/drive/MyDrive/CSE499/numpy/malign_train.npy')
benign_test = np.load('/content/drive/MyDrive/CSE499/numpy/benign_test.npy')
malign_test = np.load('/content/drive/MyDrive/CSE499/numpy/malign_test.npy')
print('Done Loaded :)')

Done Loaded :)


In [6]:
#Shape of our dataset
print(f'Benign Train:',benign_train.shape)
print(f'Malignant Train',malign_train.shape)
print(f'Benign Test:',benign_test.shape)
print(f'Malignant Test',malign_test.shape)

Benign Train: (3500, 224, 224, 3)
Malignant Train (3496, 224, 224, 3)
Benign Test: (1500, 224, 224, 3)
Malignant Test (1500, 224, 224, 3)


In [7]:
benign_train_label = np.zeros(len(benign_train),dtype=float)
malign_train_label = np.ones(len(malign_train),dtype=float)
benign_test_label = np.zeros(len(benign_test),dtype=float)
malign_test_label = np.ones(len(malign_test),dtype=float) 

X_train = np.concatenate((benign_train, malign_train), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)
X_test = np.concatenate((benign_test, malign_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

In [8]:
print(f'X train shape:',X_train.shape)
print(f'X test shape:',X_test.shape)
print(f'Y train shape:',Y_train.shape)
print(f'Y test shape:',Y_test.shape)

X train shape: (6996, 224, 224, 3)
X test shape: (3000, 224, 224, 3)
Y train shape: (6996, 2)
Y test shape: (3000, 2)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

In [10]:
print(f'x train shape:',x_train.shape)
print(f'x val shape:',x_val.shape)
print(f'y train shape:',y_train.shape)
print(f'y val shape:',y_val.shape)

x train shape: (5596, 224, 224, 3)
x val shape: (1400, 224, 224, 3)
y train shape: (5596, 2)
y val shape: (1400, 2)


In [11]:
from tensorflow.keras.applications import VGG16

def vgg16_model(backbone , lr = 1e-4):
  modelvgg16 = Sequential()
  modelvgg16.add(backbone)
  modelvgg16.add(layers.GlobalAveragePooling2D())
  modelvgg16.add(layers.Dropout(0.5))
  modelvgg16.add(layers.BatchNormalization())
  modelvgg16.add(layers.Dense(2 , activation='softmax'))
  
  modelvgg16.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
  return modelvgg16


vgg16 = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224 , 224 , 3)
)
# call the model 
modelvgg16 = vgg16_model(vgg16 , lr = 1e-4)
modelvgg16.build((None, 224, 224, 3))
modelvgg16.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                         

In [12]:
# Train the student model with MobileNet
epochs = 20
batch_size = 64
student_his_vgg16 = modelvgg16.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/20
88/88 [==============================] - 242s 2s/step - loss: 0.2300 - accuracy: 0.9160 - val_loss: 0.1541 - val_accuracy: 0.9514
Epoch 2/20
88/88 [==============================] - 156s 2s/step - loss: 0.1743 - accuracy: 0.9375 - val_loss: 0.1305 - val_accuracy: 0.9521
Epoch 3/20
88/88 [==============================] - 156s 2s/step - loss: 0.1537 - accuracy: 0.9457 - val_loss: 0.1754 - val_accuracy: 0.9329
Epoch 4/20
88/88 [==============================] - 156s 2s/step - loss: 0.1421 - accuracy: 0.9475 - val_loss: 0.1568 - val_accuracy: 0.9514
Epoch 5/20
88/88 [==============================] - 156s 2s/step - loss: 0.1339 - accuracy: 0.9500 - val_loss: 0.1464 - val_accuracy: 0.9521
Epoch 6/20
88/88 [==============================] - 156s 2s/step - loss: 0.1467 - accuracy: 0.9455 - val_loss: 0.1193 - val_accuracy: 0.9593
Epoch 7/20
88/88 [==============================] - 156s 2s/step - loss: 0.1317 - accuracy: 0.9510 - val_loss: 0.1066 - val_accuracy: 0.9586
Epoch 8/20
88

In [13]:
Y_val_pred = modelvgg16.predict(x_val)
print(f'The VGG16 Accuracy on the Validation Set:',accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1)))

The VGG16 Accuracy on the Validation Set: 0.9442857142857143


In [14]:
#Now let's check my Y_test values
print(f'My Y_test values are:\n' ,Y_test)

My Y_test values are:
 [[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [15]:
#Now let's check my predcited values from X_test dataset	
import timeit

start = timeit.default_timer()
#Your statements here
y_pred = modelvgg16.predict(X_test)
print(f'My predicted Y_test values are:\n' ,y_pred)

stop = timeit.default_timer()
print('\nTime: ',stop - start,'sec')

My predicted Y_test values are:
 [[9.99813616e-01 1.86449106e-04]
 [1.05976366e-01 8.94023657e-01]
 [9.97825503e-01 2.17447337e-03]
 ...
 [9.99116600e-01 8.83420405e-04]
 [9.99991059e-01 8.96148049e-06]
 [1.19898088e-01 8.80101919e-01]]

Time:  22.8197354670001 sec


In [16]:
#Now let's check the accuracy between the original & predicted (Y_test , y_pred)
print(f'My accuracy on Student model with VGG16 on the Test set is:',accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

My accuracy on Student model with VGG16 on the Test set is: 0.9793333333333333


In [17]:
# Print the classification report
print(classification_report( np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1500
           1       1.00      0.96      0.98      1500

    accuracy                           0.98      3000
   macro avg       0.98      0.98      0.98      3000
weighted avg       0.98      0.98      0.98      3000

